In [1]:
# !pip install torch

In [2]:
# Regular imports
import matplotlib.pyplot as plt
import torch

from torch import nn
import numpy as np 


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


'cuda'

In [4]:
import os
os.environ["TORCH_USE_CUDA_DSA"] = "1"

# Import PyTorch and other libraries




In [5]:
data_path = "/kaggle/input/majortry6/labeled_data.csv"

In [6]:
import numpy as np
import pickle
import pandas as pd
pd.options.mode.copy_on_write = True

data = pd.read_csv(data_path)
df = pd.DataFrame(data)
print(df.columns)
df.columns = ['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither',
       'classk', 'tweet']

Index(['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither',
       'class', 'tweet'],
      dtype='object')


In [7]:
df["classk"] = df["classk"].map({2:1, 1:0, 0:0}) 
# label = 0 = hate
# label = 1 = not_hate

In [8]:
tweets = df.tweet
res = []

for i in range(len(tweets)):
    splitt = tweets[i].split(":")
    if len(splitt) == 1:
        res.append(splitt[0])
    else:
        res.append(splitt[-1])
        
tweets = res

# print(tweets)

In [9]:
hate_words = ['Moist', 'Cunt', 'Panties', 'Fuck', 'Hate', 'Nigger', 'Pussy', 'Ass', 'Motherfucker', 'Bitch', 'Damn']
not_hate_words = ['Love', 'Peace', 'Kindness', 'Happiness', 'Respect', 'Friendship',
'Appreciation', 'Hope', 'Encouragement', 'Support', 'Caring']

In [10]:

from transformers import AutoTokenizer
from transformers import BertModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer("I love Pokemon")
# # tokenizer.decode(tokenizer("I love Pokemon")['input_ids'])
# from transformers import BertTokenizer, BertForSequenceClassification
# import torch
# from torch.utils.data import DataLoader, TensorDataset

# # Load pre-trained model and tokenizer
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# # Tokenize and prepare data
# input_texts = ["your input sentences"]
# labels = [0, 1, ...]  # corresponding labels

# tokenized_inputs = tokenizer(input_texts, truncation=True, padding=True, return_tensors="pt")

# # Create DataLoader
# dataset = TensorDataset(tokenized_inputs['input_ids'], tokenized_inputs['attention_mask'], torch.tensor(labels))
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# # Set up optimizer and loss function
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
# loss_fn = torch.nn.CrossEntropyLoss()

# # Fine-tuning loop
# for epoch in range(num_epochs):
#     for batch in dataloader:
#         inputs, attention_mask, target = batch
#         outputs = model(inputs, attention_mask=attention_mask)
#         loss = loss_fn(outputs.logits, target)
        
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

# # Evaluate on a test set
# # (Prepare the test set similarly and use model.eval() before inference)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

{'input_ids': [101, 1045, 2293, 20421, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [11]:
model = BertModel.from_pretrained("bert-base-uncased")

inputs = tokenizer("Hello my dog is cute", return_tensors="pt")
print(inputs)

outputs = model(**inputs)

print(len(outputs.last_hidden_state[0])) # => no. of tokens + [CLS] + [SEP]
print(len(outputs.last_hidden_state[0][1])) # => hidden dim size
print(len(outputs.last_hidden_state[0][2])) # => hidden dim size

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

{'input_ids': tensor([[  101,  7592,  2026,  3899,  2003, 10140,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}
7
768
768


In [12]:
temp = []
model = BertModel.from_pretrained("bert-base-uncased")

for word in hate_words:
    inputs = tokenizer(word, return_tensors="pt")
#     print(inputs)
    outputs = model(**inputs)
    outputs = torch.mean(outputs.last_hidden_state[0][1:len(outputs.last_hidden_state[0])], dim=0)
    temp.append(outputs.tolist())

# temp = temp[1:]
# print(hate_space)
# print(len(temp)) # => no. of words = no. of rows
# print(len(temp[0])) # => no. of columns in each row = embedding vector size

# hate_space = temp.clone().detach().requires_grad_(True)
hate_space =torch.tensor(temp).to(device).requires_grad_()

hate_space.is_leaf
hate_space.dtype
# hate_space
# hate_space.requires_grad

torch.float32

In [13]:
temp = []
model = BertModel.from_pretrained("bert-base-uncased")

# with torch.no_grad():
for word in not_hate_words:
    inputs = tokenizer(word, return_tensors="pt")
#     print(inputs)
    outputs = model(**inputs)
    outputs = torch.mean(outputs.last_hidden_state[0][1:len(outputs.last_hidden_state[0])], dim=0)
    temp.append(outputs.tolist())

# temp = temp[1:]
# print(hate_space)
# print(len(temp)) # => no. of words = no. of rows
# print(len(temp[0])) # => no. of columns in each row = embedding vector size


not_hate_space = torch.tensor(temp).to(device).requires_grad_()

print(not_hate_space.is_leaf)

# not_hate_space
# not_hate_space.requires_grad

True


In [14]:
# Trial matmul

t1 = torch.randn(11, 768)
t2 = torch.randn(4, 23, 768)

print(torch.transpose(t2, 1, 2).shape)

torch.matmul(t1, torch.transpose(t2, 1, 2)).shape

torch.Size([4, 768, 23])


torch.Size([4, 11, 23])

In [15]:
print("hate_space.grad:", hate_space.grad)
print("not_hate_space.grad:", not_hate_space.grad)

hate_space.grad: None
not_hate_space.grad: None


In [16]:
class Custom_Dataset(torch.utils.data.Dataset):
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        
    def __getitem__(self, index: int) -> (torch.Tensor, int): # => (X, y)
        
        X = self.data[index]
        y = self.labels[index]

        return (X, y)
    
    
    def __len__(self) -> int:
        
        return len(self.data)

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tweets, list(df.classk), test_size=0.2, shuffle=True)

print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

19826 19826
4957 4957


In [18]:
train_data = Custom_Dataset(data=X_train, labels=y_train)
test_data = Custom_Dataset(data=X_test, labels=y_test)

train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=3, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=1, shuffle=False)


In [19]:
import random
l=[[random.uniform(-1,1)] for i in range(11)]
l

[[0.7334505391540922],
 [0.9639288706936986],
 [0.9904389077476907],
 [0.23836832180003165],
 [-0.20956092472101506],
 [0.4108487257585165],
 [-0.806316274849916],
 [-0.5188282397948782],
 [0.07074709874099994],
 [0.2910703953056517],
 [0.04724768956072323]]

In [20]:
# next(iter(test_loader)) 
tensor_shape = (11,1)
hate_space_weights = torch.tensor(l).to(device).requires_grad_()
not_hate_space_weights = torch.tensor(l).to(device).requires_grad_()

print(hate_space_weights.device,not_hate_space_weights.is_leaf)

cuda:0 True


In [21]:
hate_space_weights.is_leaf

True

In [22]:
import torch.nn.functional as F

class Model0(nn.Module):

    def __init__(self, model, tokenizer):
        super().__init__()
        self.tokenizer = tokenizer
        self.model = model

    def forward(self, X):
        inputs = self.tokenizer(X, return_tensors="pt").to(device)
        outputs = self.model(**inputs)
        outputs = outputs.last_hidden_state.mean(1).transpose(0, 1)

        col_norms = torch.norm(outputs, p=2, dim=0, keepdim=True)
        outputs = outputs / col_norms
        
        Psk_hate = torch.matmul(torch.matmul(hate_space/torch.norm(hate_space,p=2, dim=0, keepdim=True),outputs).transpose(0, 1), hate_space_weights)
        Psk_not_hate = torch.matmul(torch.matmul(not_hate_space/torch.norm(not_hate_space,p=2, dim=0, keepdim=True),outputs).transpose(0, 1), not_hate_space_weights)

        # Construct the output tensor with requires_grad=True (assuming Psk_hate and Psk_not_hate have gradients)
        output_tensor = torch.cat((Psk_hate, Psk_not_hate), dim=0).requires_grad_()

        return output_tensor


In [23]:
model0 = Model0(model, tokenizer)

# X, y = next(iter(train_loader))
# Psk_hate, Psk_not_hate = model0(X)

# Semi-SSM

In [24]:
def L_inter(spaces):
    
    means = [torch.mean(spaces[i]) for i in range(spaces.shape[0])]
    
    loss = torch.tensor(0.0, requires_grad=True)
    
    for k in range(len(means)):
        cur = torch.tensor(0.0, requires_grad=True)
        for l in range(len(means)):
            if l == k:
                continue
            cur = cur + (1 / (1 - ((means[k] * means[l]))))
        
        loss = loss + cur
    
    return loss
            
            

def L_intra(spaces):
    loss=torch.tensor(0.0, requires_grad=True)
    for k in range(len(spaces)):
        loss=loss+1/torch.var(spaces[k], dim=0)
    
    return torch.sum(loss)/loss.shape[0]




In [25]:
L_inter(torch.concat((torch.tensor([[0.33, 0.54, 0.37], [0.72, 0.74, 0.31], [0.61, 0.26, 0.56]], dtype=torch.float32), 
       torch.tensor([[0.22, 0.32, 0.15], [0.12, -0.08, 0.45], [0.14, 0.28, -0.11]], dtype=torch.float32))))

tensor(33.6994, grad_fn=<AddBackward0>)

In [26]:
# temp = torch.tensor(10.3, requires_grad=True)

# temp

In [27]:
# res = [temp, temp]

# loss = 0

# loss += res[0] * res[1]

# loss

In [28]:
print(hate_space,not_hate_space)

tensor([[ 0.8009,  0.2601,  0.2316,  ...,  0.0455,  0.0118, -0.0370],
        [ 0.1845, -0.0866,  0.1542,  ..., -0.2302, -0.3558, -0.2348],
        [ 0.6341,  0.0180, -0.3330,  ...,  0.1851, -0.3067, -0.4023],
        ...,
        [ 0.8014,  0.2637,  0.2895,  ...,  0.1221,  0.1728, -0.4732],
        [ 0.5409,  0.4092, -0.2325,  ...,  0.1790, -0.2709, -0.0346],
        [ 0.5508,  0.1983,  0.0521,  ...,  0.3905, -0.0753, -0.5880]],
       device='cuda:0', requires_grad=True) tensor([[ 0.5933,  0.2685, -0.0267,  ..., -0.4845, -0.4192, -0.3698],
        [ 0.2493,  0.1343, -0.1606,  ..., -0.1305, -0.3765, -0.3898],
        [ 0.0551, -0.1996, -0.1785,  ..., -0.0520, -0.1908, -0.2057],
        ...,
        [ 0.2502, -0.2729, -0.2597,  ...,  0.1696, -0.5105, -0.0785],
        [ 0.2900, -0.2058, -0.1541,  ..., -0.0185, -0.5686,  0.0864],
        [ 0.2823, -0.0846, -0.1393,  ...,  0.1534, -0.2074, -0.2804]],
       device='cuda:0', requires_grad=True)


In [29]:
print(hate_space_weights,not_hate_space_weights)

tensor([[ 0.7335],
        [ 0.9639],
        [ 0.9904],
        [ 0.2384],
        [-0.2096],
        [ 0.4108],
        [-0.8063],
        [-0.5188],
        [ 0.0707],
        [ 0.2911],
        [ 0.0472]], device='cuda:0', requires_grad=True) tensor([[ 0.7335],
        [ 0.9639],
        [ 0.9904],
        [ 0.2384],
        [-0.2096],
        [ 0.4108],
        [-0.8063],
        [-0.5188],
        [ 0.0707],
        [ 0.2911],
        [ 0.0472]], device='cuda:0', requires_grad=True)


In [30]:
model0.to(device)
# hate_space = hate_space.to(device)
# not_hate_space = not_hate_space.to(device)
# hate_space_weights = hate_space_weights.to(device)
# not_hate_space_weights = not_hate_space_weights.to(device)
print(hate_space.is_leaf)
optimizer = torch.optim.Adam(params=[hate_space, not_hate_space, hate_space_weights, not_hate_space_weights], lr=0.01)
model0.train()
#criterion = nn.CrossEntropyLoss()
model0.train()
# with torch.inference_mode():
cur=0.1
prev=0
pprev=0
for _ in range(3) :
    for i, batch in enumerate(train_loader):
#         if i == 1:
#             break
        if i % 2000 == 0:
            print(i)

        x, Y = batch
        loss = 0

        for X, y in zip(x, Y):
            y_preds = model0(X)
#             print(y_preds)

            # Use long instead of float for target labels
            #print(hate_space.requires_grad)
#             print(y)
#             target = torch.tensor([y,1-y]).long().to(device)
# #             print(target)
#             loss += F.binary_cross_entropy_with_logits(y_preds.squeeze(), target.float())
            loss+=F.cross_entropy(y_preds.view(1, -1).to(device), torch.tensor([y]).to(device))
            #loss=torch.sum(y_preds)
#             print(loss)
        loss /= len(x)
        loss += 0.7*L_inter(torch.cat((hate_space, not_hate_space))) + 0.5*L_intra(torch.stack([hate_space, not_hate_space]))
            
        optimizer.zero_grad()
#         print("Before backward pass:")
#         print("hate_space.grad:", hate_space.grad)
#         print("not_hate_space.grad:", not_hate_space.grad)
        loss.backward()
        optimizer.step()
#         print("\nAfter backward pass:")
#         print("hate_space.grad:", hate_space.grad)
#         print("not_hate_space.grad:", not_hate_space.grad)
#         print(not_hate_space_weights.grad)
    correct = 0
    total = 1000

    true = []
    preds = []

    with torch.inference_mode():
        for i, batch in enumerate(test_loader):
            if i // 1000 == 1:
                break
            X, y = batch
            Psk_hate, Psk_not_hate = model0(X)

            y_predict = torch.argmax(torch.tensor([Psk_hate.mean().item(), Psk_not_hate.mean().item()]))

            true.extend(y)
            preds.append(y_predict)

            curr = torch.eq(y_predict, y)

            correct += torch.sum(curr)

    print(f"Test Accuracy: {correct/total}")
    pprev=prev
    prev=cur
    cur=correct/total
    print(f"Test Accuracy: {torch.sum(torch.eq(torch.tensor(true), torch.tensor(preds))) / total}")  




True
0
2000
4000
6000
Test Accuracy: 0.875
Test Accuracy: 0.875
0
2000
4000
6000
Test Accuracy: 0.8740000128746033
Test Accuracy: 0.8740000128746033
0
2000
4000
6000
Test Accuracy: 0.8859999775886536
Test Accuracy: 0.8859999775886536


In [31]:
print(hate_space_weights,not_hate_space_weights)

tensor([[ 0.7345],
        [ 1.9713],
        [ 1.9825],
        [ 0.9015],
        [-0.1902],
        [ 0.6732],
        [-2.2366],
        [-0.5141],
        [ 0.5536],
        [ 1.1785],
        [-0.0204]], device='cuda:0', requires_grad=True) tensor([[ 1.6272],
        [ 1.7508],
        [ 2.0449],
        [ 0.0073],
        [-0.9467],
        [ 0.4874],
        [-2.3485],
        [-1.5678],
        [-0.4271],
        [ 0.5754],
        [-0.4198]], device='cuda:0', requires_grad=True)


In [32]:
print(hate_space,not_hate_space)

tensor([[ 0.4803,  0.2057,  1.3476,  ..., -2.3622, -0.0763,  0.2768],
        [-1.4151, -1.0532,  1.1992,  ..., -2.9778, -1.5113, -0.3248],
        [-0.0334, -0.8115,  0.2044,  ..., -1.7865, -1.4128, -1.1949],
        ...,
        [ 0.6699,  0.6601,  1.9514,  ..., -0.6066,  0.6660, -2.6002],
        [-2.3129,  0.7285, -0.5741,  ..., -1.2018, -3.2721,  0.2426],
        [ 2.4585,  2.2479,  2.3060,  ...,  0.6918,  1.1279, -1.6171]],
       device='cuda:0', requires_grad=True) tensor([[  2.6559,   1.8290,  -0.9458,  ...,   2.0247,  -1.5178,  -4.4388],
        [  0.9405,   1.2038,  -1.4698,  ...,   3.7502,   4.9180,  -2.1707],
        [  0.2732,  -1.2368,  -2.0090,  ...,   6.4301,   1.8691,   0.1994],
        ...,
        [-16.6772,  -2.1928,  -1.9449,  ...,  -2.6234,  -5.7503,  13.9322],
        [ -0.7610,  -5.7348,  -0.5972,  ...,   5.6868,  -0.6098,   2.1769],
        [-14.2699,  -5.2073,   5.6502,  ...,  -0.1315,   2.4860,  -5.7929]],
       device='cuda:0', requires_grad=True)


In [33]:
torch.concat((hate_space, not_hate_space)).shape

torch.Size([22, 768])

In [34]:
correct = 0
total = len(y_test)

true = []
preds = []

with torch.inference_mode():
    for i, batch in enumerate(test_loader):
        if i < 1000 == 0:
            pass
        X, y = batch
        Psk_hate, Psk_not_hate = model0(X)

        y_predict = torch.argmax(torch.tensor([Psk_hate.mean().item(), Psk_not_hate.mean().item()]))

        true.extend(y)
        preds.append(y_predict)

        cur = torch.eq(y_predict, y)

        correct += torch.sum(cur)

print(f"Test Accuracy: {correct/total}")
print(f"Test Accuracy: {torch.sum(torch.eq(torch.tensor(true), torch.tensor(preds))) / total}")       

Test Accuracy: 0.8882388472557068
Test Accuracy: 0.8882388472557068


In [35]:
from sklearn.metrics import precision_score, recall_score

print(precision_score(y_test, preds, average='macro'))
print(recall_score(y_test, preds, average='macro'))

0.8361480613495106
0.7236777389277389
